In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm


In [3]:
sp_tech = pd.read_excel("C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/sinais_ind_tecnicos.xlsx")


In [4]:
sp_tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Read the Excel file into a pandas DataFrame
sp_tech = pd.read_excel("C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/sinais_ind_tecnicos.xlsx")

# Handle NaN values for 'Adj Close', perhaps with forward fill or dropna:
sp_tech['Adj Close'].fillna(method='ffill', inplace=True)

# Calculate the percentage returns of the 'Adj Close' column
sp_tech['return'] = sp_tech['Adj Close'].pct_change().fillna(0)

# Initialization
n = 14  # Number of technical indicators
t = len(sp_tech)
initial_period = 150  # Initial period for the first regression
summaries = []  # To store summary objects for later use

# Arrays to store predictions and errors
predictions = np.zeros((t - initial_period, n))
errors = np.zeros((t - initial_period, n))

# Loop for rolling regression
for i in range(n):
    # The dependent variable (Y)
    Y = sp_tech['return'].values
    
    # Loop to perform rolling regression and predictions
    for current_t in range(initial_period, len(Y)):
        # The independent variable (X)
        X = sp_tech.iloc[0:current_t, 3 + i].values  # Use data from the beginning up to the current time
        X = sm.add_constant(X)  # Add the intercept
        
        # Perform regression
        model = sm.OLS(Y[0:current_t], X).fit()

        # Make prediction for the next value if we're not at the end of the dataset
        if current_t < len(Y) - 1:
            
            next_x = np.array([[1, sp_tech.iloc[current_t + 1, 3 + i]]])  # Prepare next X value as a 2D array 
            predicted_value = model.predict(next_x)[0]
            predictions[current_t - initial_period, i] = predicted_value
            
            # Calculate and store the error of the prediction
            actual_value = Y[current_t + 1]
            error = actual_value - predicted_value
            errors[current_t - initial_period, i] = error
        #print(current_t)
    
    summaries.append(model.summary())

# The 'predictions' array contains the predicted values for each technical indicator
# The 'errors' array contains the prediction errors for each technical indicator


150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399


In [6]:
summaries

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.162
 Model:                            OLS   Adj. R-squared:                  0.160
 Method:                 Least Squares   F-statistic:                     86.74
 Date:                Wed, 08 Nov 2023   Prob (F-statistic):           5.52e-19
 Time:                        08:27:06   Log-Likelihood:                 438.74
 No. Observations:                 452   AIC:                            -873.5
 Df Residuals:                     450   BIC:                            -865.3
 Df Model:                           1                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------